In [1]:
%matplotlib widget
from util import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab, from_nx_to_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
from matplotlib import colors
from copy import deepcopy,copy
from collections import Counter
import cv2 
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage.morphology import thin
from skimage import data, filters
from random import choice
import scipy.sparse

In [4]:
date = '0701_1557'
date_plate = f'/2020{date}'
plate = 13
plate_str = f'_Plate{plate}'
path_tile='//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Img/TileConfiguration.registered.txt'
tileconfig = pd.read_table(path_tile,sep=';',skiprows=4,header=None,converters={2 : ast.literal_eval},skipinitialspace=True)
ims = []
for name in tileconfig[0]:
#     ims.append(imageio.imread('//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Img/'+name))
    ims.append(np.load(f'Temp\{name}.npy'))

In [5]:
xs =[c[0] for c in tileconfig[2]]
ys =[c[1] for c in tileconfig[2]]
dim = (int(np.max(ys)-np.min(ys))+4096,int(np.max(xs)-np.min(xs))+4096)
import scipy.sparse
shape = (3000,4096)
skel = np.zeros(dim,dtype=np.uint8)
for index,im in enumerate(ims):
    im_cropped = im
    im_blurred =cv2.blur(im_cropped, (200, 200))
    im_back_rem = (im_cropped+1)/(im_blurred+1)
    im_back_rem = cv2.normalize(im_back_rem, None, 0, 255, cv2.NORM_MINMAX)
    im_back_rem_inv = cv2.normalize(255-im_back_rem, None, 0, 255, cv2.NORM_MINMAX)
#     frangised = frangi(im_back_rem,sigmas=range(1,20,4))
#     frangised = cv2.normalize(frangised, None, 0, 255, cv2.NORM_MINMAX)
#     transformed = cv2.normalize(frangised-im_back_rem, None, 0, 255, cv2.NORM_MINMAX)
    transformed = cv2.normalize(-im_back_rem, None, 0, 255, cv2.NORM_MINMAX)
    low = 180
    high = 200
    lowt = (transformed > low).astype(int)
    hight = (transformed > high).astype(int)
    hyst = filters.apply_hysteresis_threshold(transformed, low, high)
    kernel = np.ones((3,3),np.uint8)
    dilation = cv2.dilate(hyst.astype(np.uint8) * 255,kernel,iterations = 1)
    for i in range(3):
        dilation=cv2.erode(dilation.astype(np.uint8) * 255,kernel,iterations = 1)
        dilation = cv2.dilate(dilation.astype(np.uint8) * 255,kernel,iterations = 1)
    dilated = dilation>0
    sparse_dilated = scipy.sparse.csc_matrix(dilated)
#     np.save(f'Temp\dilated{tileconfig[0][i]}',dilated)
    boundaries = int(tileconfig[2][index][0]-np.min(xs)),int(tileconfig[2][index][1]-np.min(ys))
#     print(boundaries)
#     print(skel[boundaries[1]:boundaries[1]+shape[0],boundaries[0]:boundaries[0]+shape[1]].shape,sparse_dilated.shape)
    if np.mean(sparse_dilated)<=0.5:
        skel[boundaries[1]:boundaries[1]+shape[0],boundaries[0]:boundaries[0]+shape[1]] += sparse_dilated
#     skeletonized = thin(dilated)
#     break

In [ ]:
for name in tileconfig[0]:
#     ims.append(imageio.imread('//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Img/'+name))
    im=np.load(f'Temp\{name}.npy')
    if np.mean(im)<=100:
        print(name)
#     ims.append(np.load(f'Temp\{name}.npy'))

In [ ]:
for name in tileconfig[0]:
#     ims.append(imageio.imread('//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Img/'+name))
    im=np.load(f'Temp\{name}.npy')
    if name == 'Img_r07_c12.tif':
        print(np.mean(im))
        plt.close('all')
        fig=plt.figure(figsize=(10,9))
        ax = fig.add_subplot(111)
        ax.imshow(im)
        imi = im

In [12]:
import scipy.sparse
for index,im in enumerate(ims):
    im_cropped = im
    im_blurred =cv2.blur(im_cropped, (200, 200))
    im_back_rem = (im_cropped+1)/(im_blurred+1)
    im_back_rem = cv2.normalize(im_back_rem, None, 0, 255, cv2.NORM_MINMAX)
    im_back_rem_inv = cv2.normalize(255-im_back_rem, None, 0, 255, cv2.NORM_MINMAX)
#     frangised = frangi(im_back_rem,sigmas=range(1,20,4))
#     frangised = cv2.normalize(frangised, None, 0, 255, cv2.NORM_MINMAX)
#     transformed = cv2.normalize(frangised-im_back_rem, None, 0, 255, cv2.NORM_MINMAX)
    transformed = cv2.normalize(-im_back_rem, None, 0, 255, cv2.NORM_MINMAX)
    low = 100
    high = 200
    lowt = (transformed > low).astype(int)
    hight = (transformed > high).astype(int)
    hyst = filters.apply_hysteresis_threshold(transformed, low, high)
    kernel = np.ones((3,3),np.uint8)
    dilation = cv2.dilate(hyst.astype(np.uint8) * 255,kernel,iterations = 1)
    for i in range(3):
        dilation=cv2.erode(dilation.astype(np.uint8) * 255,kernel,iterations = 1)
        dilation = cv2.dilate(dilation.astype(np.uint8) * 255,kernel,iterations = 1)
    dilated = dilation>0
    sparse_dilated = scipy.sparse.csc_matrix(dilated)
#     np.save(f'Temp\dilated{tileconfig[0][i]}',dilated)
    scipy.sparse.save_npz(f'Temp/sparse_matrix{tileconfig[0][index][:-4]}.npz', sparse_dilated)
#     skeletonized = thin(dilated)
#     break

In [6]:
np.sum(skel>0),np.sum(skel<=0)

(13091473, 1296470273)

In [ ]:
skeletonized = cv2.ximgproc.thinning(np.array(255*(skel>0),dtype=np.uint8))

In [ ]:
import cProfile
cProfile.run('cv2.ximgproc.thinning(np.array(255*(skel>0),dtype=np.uint8))')

In [8]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.imshow(skeletonized)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
tileconfig[0][3][:-4]

'Img_r01_c04'

In [5]:
xs =[c[0] for c in tileconfig[2]]
ys =[c[1] for c in tileconfig[2]]

In [6]:
dim = (int(np.max(ys)-np.min(ys))+4096,int(np.max(xs)-np.min(xs))+4096)

In [35]:
skel = np.zeros(dim,dtype=np.uint8)

In [36]:
import sys
sys.getsizeof(skel)/(10**6)

1309.561858

In [38]:
shape = (3000,4096)
for index,im in enumerate(ims):
    sparse_dilated = scipy.sparse.load_npz(f'Temp/sparse_matrix{tileconfig[0][index][:-4]}.npz').todense()
    boundaries = int(tileconfig[2][index][0]-np.min(xs)),int(tileconfig[2][index][1]-np.min(ys))
#     print(boundaries)
#     print(skel[boundaries[1]:boundaries[1]+shape[0],boundaries[0]:boundaries[0]+shape[1]].shape,sparse_dilated.shape)
    skel[boundaries[1]:boundaries[1]+shape[0],boundaries[0]:boundaries[0]+shape[1]] += sparse_dilated

In [7]:
skel = skel>0


NameError: name 'skel' is not defined

In [52]:
np.sum(skeletonized)

0

In [41]:
skel.dtype=np.uint8

In [27]:
sys.getsizeof(skel)/(10**6)

1309.561858

In [72]:
image = cv2.imread("OpenCV.png")
thinned = cv2.ximgproc.thinning(np.array(255-255*(cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)>0),dtype=np.uint8))

In [73]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.imshow(thinned)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.imshow(skeletonized[10000:20000,10000:20000])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
img1 = skel.copy()
# Structuring Element
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
# Create an empty output image to hold values
thin = np.zeros(img1.shape,dtype='uint8') 
while (cv2.countNonZero(img1)!=0):
    # Erosion
    erode = cv2.erode(img1,kernel)
    # Opening on eroded image
    opening = cv2.morphologyEx(erode,cv2.MORPH_OPEN,kernel)
    # Subtract these two
    subset = erode - opening
    # Union of all previous sets
    thin = cv2.bitwise_or(subset,thin)
    # Set the eroded image for next iteration
    img1 = erode.copy()

In [28]:
skeletonized = thin(skel)

KeyboardInterrupt: 

In [111]:
scipy.sparse.save_npz(f'Temp/sparse_matrix{tileconfig[0][i]}.npz', sparse_dilated)


In [56]:
values = [np.mean(im) for im in ims]

In [57]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.hist(values)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([33.,  2.,  4.,  5.,  8.,  8., 10.,  7., 32., 41.]),
 array([  0.        ,  15.46855857,  30.93711714,  46.40567571,
         61.87423428,  77.34279285,  92.81135142, 108.27990999,
        123.74846855, 139.21702712, 154.68558569]),
 <BarContainer object of 10 artists>)

In [107]:
plt.close('all')
im = choice(ims)
print(np.mean(im))
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.imshow(im)

11.942084879557292


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
im_cropped = im
im_blurred =cv2.blur(im_cropped, (200, 200))
im_back_rem = (im_cropped+1)/(im_blurred+1)
im_back_rem = cv2.normalize(im_back_rem, None, 0, 255, cv2.NORM_MINMAX)
im_back_rem_inv = cv2.normalize(255-im_back_rem, None, 0, 255, cv2.NORM_MINMAX)
frangised = frangi(im_back_rem,sigmas=range(1,20,4))
frangised = cv2.normalize(frangised, None, 0, 255, cv2.NORM_MINMAX)
transformed = cv2.normalize(frangised-im_back_rem, None, 0, 255, cv2.NORM_MINMAX)
low = 100
high = 200
lowt = (transformed > low).astype(int)
hight = (transformed > high).astype(int)
hyst = filters.apply_hysteresis_threshold(transformed, low, high)
kernel = np.ones((3,3),np.uint8)
dilation = cv2.dilate(hyst.astype(np.uint8) * 255,kernel,iterations = 1)
for i in range(3):
    dilation=cv2.erode(dilation.astype(np.uint8) * 255,kernel,iterations = 1)
    dilation = cv2.dilate(dilation.astype(np.uint8) * 255,kernel,iterations = 1)
dilated = dilation>0
skeletonized = thin(dilated)

In [108]:
im_cropped = im
im_blurred =cv2.blur(im_cropped, (200, 200))
im_back_rem = (im_cropped+1)/(im_blurred+1)
im_back_rem = cv2.normalize(im_back_rem, None, 0, 255, cv2.NORM_MINMAX)

In [109]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.imshow(im_back_rem)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [55]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.imshow(skeletonized)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …